## In this Notebook, we will create a basic Q&A chain and Agent over a SQL database

### Architecture

At a high-level, the steps of any SQL chain and agent are:

- `Convert question to SQL query using the LLM Model.`
- `Execute the SQL query.`
- `getting the Model to respond to user input using the query results.`

In [3]:
%pip install langchain langchain_openai langchain_experimental python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 1.2 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
llm = ChatOpenAI(temperature=0)

In [4]:
host = 'localhost'
port = '3306'
username = 'root'
password = 'xxxxx'
database_schema = 'analytics'
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=2)
chain = create_sql_query_chain(llm, db)

In [5]:
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT count(*) FROM customer LIMIT 10;")

mysql
['customer', 'employee', 'inventoryitem', 'itemsize', 'manufacturer', 'product', 'purchase', 'purchaseitem', 'sale', 'saleitem', 'your_table_name']


'[(4417,)]'

In [6]:
response = chain.invoke({"question": "How many employees are there"})
response

'SELECT COUNT(employeeid) AS total_employees FROM employee;'

In [7]:
db.run(response)

'[(21,)]'

In [8]:
response = chain.invoke({"question": "How many customers are from Bronx"})
print(response)
print(db.run(response))

SELECT COUNT(*) AS num_customers
FROM customer
WHERE city = 'Bronx';
[(9,)]


In [9]:
response = chain.invoke({"question": "Which state are customers most from?"})
print(response)
print(db.run(response))

SELECT state, COUNT(*) AS total_customers
FROM customer
GROUP BY state
ORDER BY total_customers DESC
LIMIT 1;
[('CA', 375)]


In [10]:
response = chain.invoke({"question": "Give me the top 5 customers who paid the highest shipping charges"})
print(response)
print(db.run(response))

SELECT c.customerid, c.firstname, c.lastname, c.streetaddress, c.city, c.state, c.postalcode, c.country, c.phone
FROM customer c
JOIN sale s ON c.customerid = s.customerid
ORDER BY s.shipping DESC
LIMIT 5;
[(16433, 'Freddie', 'Shafer', '5201 S Dorchester Ave', 'Chicago', 'IL', '60615', 'USA', '(312) 288-8190'), (12976, 'Cassie', 'Hill', '4612 Broad St #202', 'Virginia Beach', 'VA', '23462', 'USA', '(757) 490-7037'), (1762, 'Lorie', 'Obal', '132 Karen Pl', 'Steubenville', 'OH', '43953', 'USA', '(740) 264-2378'), (1096, 'Meliha', 'Handzic', '14734 4th St', 'Laurel', 'MD', '20707', 'USA', '(301) 483-7981'), (15651, 'Kathleen', 'Hartzel', '1208 Dunleaf Arc Way', 'Norcross', 'GA', '30093', 'USA', '(770) 931-3172')]
